In [1]:
import pandas as pd
import numpy as np
!pip install geopandas 
import geopandas as gpd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.4 MB/s 
     |████████████████████████████████| 16.7 MB 387 kB/s 
     |████████████████████████████████| 6.3 MB 35.0 MB/s 


In [2]:
df = pd.read_csv('gas-flare.csv')
df.head(2)

,country,flare_16,flare_17,flare_18,flare_19,flare_20,flare_21,change_20_21
0,Russia,22.37,19.92,2128.00,23.21,24.88,25.41,0.53
1,Iraq,17.73,17.84,17.82,17.91,17.37,17.80,0.43


In [3]:
pd.set_option('display.max_columns', None)
countries_geo = gpd.read_file("countries.geojson")
countries_geo.head(2)

,ADMIN,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."


In [4]:
countries_geo = countries_geo.rename(columns={'ADMIN': 'country'})
countries_geo.head(2)

,country,ISO_A3,geometry
0,Aruba,ABW,"POLYGON ((-69.99694 12.57758, -69.93639 12.531..."
1,Afghanistan,AFG,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."


In [5]:
df2 = pd.merge(df, countries_geo, how="left", left_on="country", right_on="country")
df2.head(5)

,country,flare_16,flare_17,flare_18,flare_19,flare_20,flare_21,change_20_21,ISO_A3,geometry
0,Russia,22.37,19.92,2128.00,23.21,24.88,25.41,0.53,RUS,"MULTIPOLYGON (((132.44899 42.84540, 132.44988 ..."
1,Iraq,17.73,17.84,17.82,17.91,17.37,17.80,0.43,IRQ,"POLYGON ((42.89674 37.32491, 42.93705 37.32015..."
2,Iran,16.41,17.67,17.28,13.78,13.26,17.38,4.12,IRN,"MULTIPOLYGON (((55.05437 25.86461, 55.04648 25..."
3,United States of America,8.86,9.48,14.07,17.29,11.81,8.78,-3.03,USA,"MULTIPOLYGON (((-155.60652 20.13796, -155.5863..."
4,Algeria,9.10,8.80,9.01,9.34,9.32,8.16,-1.16,DZA,"POLYGON ((8.60251 36.93951, 8.60566 36.91305, ..."


In [6]:
#Calculating percentage change
df2['pctChange'] = df2['flare_20'] - df2['flare_21'] / df2['flare_21'] * 100

In [7]:
df2.head()

,country,flare_16,flare_17,flare_18,flare_19,flare_20,flare_21,change_20_21,ISO_A3,geometry,pctChange
0,Russia,22.37,19.92,2128.00,23.21,24.88,25.41,0.53,RUS,"MULTIPOLYGON (((132.44899 42.84540, 132.44988 ...",-75.12
1,Iraq,17.73,17.84,17.82,17.91,17.37,17.80,0.43,IRQ,"POLYGON ((42.89674 37.32491, 42.93705 37.32015...",-82.63
2,Iran,16.41,17.67,17.28,13.78,13.26,17.38,4.12,IRN,"MULTIPOLYGON (((55.05437 25.86461, 55.04648 25...",-86.74
3,United States of America,8.86,9.48,14.07,17.29,11.81,8.78,-3.03,USA,"MULTIPOLYGON (((-155.60652 20.13796, -155.5863...",-88.19
4,Algeria,9.10,8.80,9.01,9.34,9.32,8.16,-1.16,DZA,"POLYGON ((8.60251 36.93951, 8.60566 36.91305, ...",-90.68


In [8]:
df2 = gpd.GeoDataFrame(df2, crs="EPSG:4326", geometry=df2.geometry)

In [9]:
df2.to_file('gas-flare.geojson', driver='GeoJSON')

In [10]:
import altair as alt

In [56]:
import altair as alt


countries = alt.Chart(countries_geo).mark_geoshape(
    fill='whitesmoke',
    stroke='gray',
    strokeWidth=0.5,
).encode(
    tooltip=['country']
).properties(
    width=500,
    height=300
)

gasFlare = alt.Chart(df2).mark_geoshape().encode(
    color='flare_21'
).transform_lookup(
    lookup='country',
    from_=alt.LookupData(df2, 'country', ['flare_21'])
).encode(
    tooltip=['country', 'flare_21']
).project(
    type='equirectangular'
).properties(
    width=500,
    height=300
)

countriesFlare =  alt.layer(countries, gasFlare, title='Gas flare levels per country in billion cubic meters'
)
countriesFlare
countriesFlare.save('gas_chart.html' )

